In [2]:
import requests as rq
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from datetime import datetime

In [108]:
urls = ['https://www.basketball-reference.com/play-index/tgl_finder.cgi?request=1&match=game&lg_id=NBA&is_playoffs=N&team_seed_cmp=eq&opp_seed_cmp=eq&year_min=2011&year_max=2020&is_range=N&game_num_type=team&order_by=date_game']

qq = 'https://www.basketball-reference.com/play-index/tgl_finder.cgi?request=1&player=&match=game&lg_id=NBA&year_min=2011&year_max=2020&team_id=&opp_id=&is_range=N&is_playoffs=N&round_id=&best_of=&team_seed=&opp_seed=&team_seed_cmp=eq&opp_seed_cmp=eq&game_num_type=team&game_num_min=&game_num_max=&game_month=&game_location=H&game_result=&is_overtime=&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=date_game&order_by_asc=&offset='
for i in range (100,11701,100):
    y=str(i)
    urls.append(qq+y)

r_bbref_3d = rq.get(urls[0])
stats_bbref_soup = BeautifulSoup(r_bbref_3d.text, 'lxml')

look_for_headers = stats_bbref_soup.table.thead.find_all('tr')
look_for_headers[:1]
headers_list = look_for_headers[1].text
headers_list = headers_list.split('\n')
headers_list = headers_list[2:-1]

for i in range(19,32):
    headers_list[i] = 'OPP '+headers_list[i]

team_stats_byrow = []
temp = []

for url in urls:
    r_bbref_3d = rq.get(url)
    stats_bbref_soup = BeautifulSoup(r_bbref_3d.text, 'lxml')
            
    every_row = stats_bbref_soup.table.tbody.find_all('tr')  
    for row in every_row:
        list_of_stats = row.find_all('td')
        for td in list_of_stats:
            stat = td.text
            temp.append(stat)
        team_stats_byrow.append(temp)
        temp = []
        
teamStats_df = pd.DataFrame(data=team_stats_byrow, columns=headers_list)
teamStats_df = teamStats_df.drop(teamStats_df.columns[2], axis=1)
teamStats_df = teamStats_df.dropna()

teamStats_df[['MP', 'FG', 'FGA', '2P', '2PA', '3P', '3PA', 'FT', 'FTA', 'PTS', 'OPP FG', 'OPP FGA', 'OPP 2P', 'OPP 2PA', 'OPP 3P', 'OPP 3PA', 'OPP FT', 'OPP FTA', 'OPP PTS']] = teamStats_df[['MP', 'FG', 'FGA', '2P', '2PA', '3P', '3PA', 'FT', 'FTA', 'PTS', 'OPP FG', 'OPP FGA', 'OPP 2P', 'OPP 2PA', 'OPP 3P', 'OPP 3PA', 'OPP FT', 'OPP FTA', 'OPP PTS']].astype(int)
teamStats_df[['FG%', '2P%', '3P%', 'FT%', 'OPP FG%', 'OPP 2P%', 'OPP 3P%', 'OPP FT%']] = teamStats_df[['FG%', '2P%', '3P%', 'FT%', 'OPP FG%', 'OPP 2P%', 'OPP 3P%', 'OPP FT%']].astype(float)
teamStats_df['Date'] = pd.to_datetime(teamStats_df['Date'])

teamStats_df 

,Date,Tm,Opp,Result,MP,FG,FGA,FG%,2P,2PA,...,OPP 2P,OPP 2PA,OPP 2P%,OPP 3P,OPP 3PA,OPP 3P%,OPP FT,OPP FTA,OPP FT%,OPP PTS
0,2020-03-11,ATL,NYK,L 131-136 (OT),265,48,106,0.453,34,65,...,39,66,0.591,11,28,0.393,25,34,0.735,136
1,2020-03-11,NYK,ATL,W 136-131 (OT),265,50,94,0.532,39,66,...,34,65,0.523,14,41,0.341,21,26,0.808,131
2,2020-03-11,DAL,DEN,W 113-97,240,42,92,0.457,30,57,...,25,44,0.568,14,39,0.359,5,8,0.625,97
3,2020-03-11,DEN,DAL,L 97-113,240,39,83,0.470,25,44,...,30,57,0.526,12,35,0.343,17,23,0.739,113
4,2020-03-11,CHO,MIA,W 109-98,240,37,82,0.451,20,48,...,24,40,0.600,15,44,0.341,5,6,0.833,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12267,2010-10-27,SAS,IND,W 122-109,240,42,83,0.506,33,63,...,33,56,0.589,8,21,0.381,19,28,0.679,109
12268,2010-10-27,TOR,NYK,L 93-98,240,36,94,0.383,33,84,...,31,64,0.484,7,24,0.292,15,17,0.882,98
12269,2010-10-26,BOS,MIA,W 88-80,240,32,69,0.464,24,53,...,19,54,0.352,8,20,0.400,18,25,0.720,80
12270,2010-10-26,LAL,HOU,W 112-110,240,40,96,0.417,31,75,...,30,71,0.423,8,20,0.400,26,28,0.929,110


In [109]:
teamStats_df.to_csv('NBA_Games_all_Decade.csv')

In [106]:
teamStats_df.columns
# teamStats_df['Date'][2]
teamStats_df['Date'] = pd.to_datetime(teamStats_df['Date'])
# type(teamStats_df['Date'][2])
teamStats_df['Date'][222]
# teamStats_df['Date'][22] - teamStats_df['Date'][222]

Timestamp('2020-02-05 00:00:00')